# Non-Ergodic Dynamics in Multiplicative Systems

In many stochastic systems, growth occurs multiplicatively rather than additively.
Such systems exhibit fundamentally different long-term behavior, where ensemble averages
diverge from time-averaged outcomes experienced by individual trajectories.

This notebook investigates the following question:

> Why do most trajectories in multiplicative systems lose value over time,
> despite the existence of rare paths with extremely high returns?

Using Monte Carlo simulation and binomial (CRR) trees, we demonstrate that:
1. Log-growth, not expected value, governs long-term outcomes.
2. Increased volatility amplifies the divergence between mean and median wealth.
3. Value in option pricing is dominated by rare, extreme states rather than typical outcomes.

The analysis proceeds in three stages: simulation of wealth processes,
analytical examination of log-growth, and state-space analysis via binomial trees.

```

Phase I: Scope, Assumptions & Interpretation Rules
├── define multiplicative system under study
├── clarify simulation-based (no external data)
├── distinguish ensemble vs time-average perspective
└── state non-goals (no market prediction, no strategy optimization)

Phase II: Multiplicative Wealth Process Setup
├── define wealth dynamics and return distributions
├── specify volatility regimes (low / medium / high)
├── define horizon lengths and number of paths
└── establish baseline expectations

Phase III: Monte Carlo Dynamics & Path-Level Behavior
├── simulate wealth trajectories across regimes
├── visualize path fan charts
├── inspect distributional spread over time
└── identify early divergence between paths

Phase IV: Ensemble vs Time-Average Outcomes
├── compute mean, median, and quantiles of wealth
├── compare ensemble average vs typical trajectory
├── analyze survival vs extinction behavior
└── demonstrate non-ergodicity empirically

Phase V: Log-Growth & Volatility Drag Analysis
├── compute expected log-growth analytically
├── relate volatility to growth degradation
├── contrast additive vs multiplicative accumulation
└── reconcile theory with simulation outcomes

Phase VI: Binomial / CRR Tree as State Space
├── construct recombining binomial (CRR) tree
├── interpret nodes as states, not probabilities
├── assign transition probabilities
└── examine state-space geometry over time

Phase VII: Probability, Value & Contribution Decomposition
├── compute probability mass per node
├── compute payoff / wealth per node
├── measure contribution to expectation
└── contrast high-probability vs high-impact states

Phase VIII: Wealth Dynamics ↔ Option Pricing Analogy
├── map wealth paths to price paths
├── relate rare extreme paths to option value
├── interpret volatility asymmetry
└── explain why tails dominate valuation

Phase IX: Synthesis & Structural Insights
├── summarize non-ergodic mechanisms observed
├── articulate volatility–survival trade-off
├── clarify limits of expectation-based reasoning
└── outline implications for stochastic modeling


```

## Import

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid", context="talk")
plt.rcParams["figure.figsize"] = (10, 6)

from src import (
    generate_returns,
    simulate_wealth,
    volatility_regimes,
    run_monte_carlo,
    wealth_distribution_stats,
    time_average_log_growth,
    ensemble_arithmetic_growth,
    compare_growth_measures,
    survival_probability,
    terminal_extinction_rate,
    terminal_log_wealth_stats,
    gini_coefficient,
    terminal_inequality,
    crr_params,
    build_crr_tree,
    node_probabilities,
    level_expectations,
    expectation_contributions,
    terminal_tail_dominance,
    run_crr_state_space,
    plot_fan_chart,
    plot_mean_median,
    plot_terminal_log_distribution,
    plot_survival_curve,
    plot_crr_contribution_heatmap,
    plot_crr_expectation,
)

# Phase I — Scope, Assumptions, and Interpretation

## 1. Multiplicative Wealth Dynamics

Trong nghiên cứu này, ta xem xét một hệ tăng trưởng **nhân** theo thời gian, trong đó giá trị tài sản được cập nhật theo quy tắc:

$$
W_{t+1} = W_t \cdot R_t
$$

với:

- $( W_t)$: tài sản tại thời điểm $( t )$
- $( R_t)$: tỷ suất sinh lợi ngẫu nhiên tại thời điểm $( t )$

Sau \( T \) bước thời gian:

$$
W_T = W_0 \prod_{t=1}^{T} R_t
$$

Dưới dạng log:

$$
\log W_T = \log W_0 + \sum_{t=1}^{T} \log R_t
$$

Biểu diễn log cho thấy **tăng trưởng dài hạn** được quyết định bởi **kỳ vọng của log-return**,
không phải bởi kỳ vọng của return thông thường.
Đây là điểm khởi đầu cho hiện tượng **non-ergodicity trong hệ nhân**.


 ## 2. Simulation-Based Methodology

Notebook này sử dụng **mô phỏng Monte Carlo thuần túy** để nghiên cứu động lực học của hệ tăng trưởng nhân.

Các đặc điểm chính:

- Không sử dụng dữ liệu thị trường thực.
- Không thực hiện dự báo giá.
- Không tối ưu chiến lược đầu tư.
- Chỉ tập trung vào **cấu trúc xác suất nội tại** của quá trình tăng trưởng ngẫu nhiên.

Cách tiếp cận này cho phép:

- tách biệt hoàn toàn yếu tố toán học khỏi nhiễu dữ liệu thực,
- quan sát trực tiếp sự phân kỳ giữa các quỹ đạo tài sản,
- kiểm chứng thực nghiệm các kết quả lý thuyết về log-growth và non-ergodicity.


## 3. Ensemble vs Time-Average Perspective

Trong các hệ cộng (*additive systems*), trung bình theo quần thể (*ensemble average*)
và trung bình theo thời gian (*time average*) thường trùng nhau.

Tuy nhiên, với hệ tăng trưởng **nhân**, hai đại lượng này khác biệt cơ bản:

- **Ensemble expectation**:

  $$
  \mathbb{E}[W_T]
  $$

  bị chi phối bởi các quỹ đạo hiếm nhưng có giá trị cực lớn.

- **Time-average growth**:

  $$
  \mathbb{E}[\log R]
  $$

  mới là đại lượng quyết định trải nghiệm dài hạn của **một quỹ đạo điển hình**.

Sự không tương đương này chính là biểu hiện của:

> **Non-ergodicity trong động lực học nhân.**


## 4. Non-Goals and Interpretation Limits

Nghiên cứu này **không nhằm mục đích**:

- dự đoán thị trường tài chính,
- đề xuất chiến lược đầu tư,
- định giá tài sản thực,
- đưa ra khuyến nghị tài chính.

Thay vào đó, mục tiêu duy nhất là:

> phân tích **cơ chế toán học và xác suất** của tăng trưởng ngẫu nhiên nhân,
> cùng các hệ quả cấu trúc như non-ergodicity, volatility drag,
> và sự chi phối của các trạng thái hiếm trong kỳ vọng.

Mọi diễn giải tài chính (nếu có) chỉ mang tính **ẩn dụ cấu trúc**,
không phải kết luận thực nghiệm về thị trường.
